In [1]:
!pip3 install requests beautifulsoup4 lxml

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://blog.gov.bc.ca/bcwildfire/page/'
news_links = []


for page in range(1, 20):
    url = f'{base_url}{page}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}")
        break

    soup = BeautifulSoup(response.text, 'lxml')
    articles = soup.find_all('div', class_='post-feed-box')

    for article in articles:
        link = article.find('h1').find('a')['href']
        news_links.append(link)

In [20]:
from datetime import datetime

def convert_to_timestamp(date_str, date_format):
    date_obj = datetime.strptime(date_str, date_format)
    timestamp = int(date_obj.timestamp())
    return timestamp

In [23]:
import pandas as pd

df = pd.DataFrame({"title": [], "date": [], "content": [], "link": []})

for link in news_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'lxml')
    title = soup.find('h1', class_="entry-title").text
    date = soup.find('time').text
    content = soup.find('div', class_='entry-content').text
    df.loc[len(df)] = {"title": title, "date": convert_to_timestamp(
        date, "%B %d, %Y"), "content": content, "link": link}


In [24]:
df.to_csv('bc_wildfire_news.csv', index=False)

,title,date,content,link
0,Campfire Prohibition Partially Rescinded in No...,1694498400,\nSMITHERS – Effective at 12:00 p.m. (noon) on...,https://blog.gov.bc.ca/bcwildfire/cat1-partial...
1,Forecasted weather conditions expected to incr...,1694325600,\nPRINCE GEORGE – Weather conditions across th...,https://blog.gov.bc.ca/bcwildfire/forecasted-w...
2,Area restriction for the Jubilee Mountain wild...,1694152800,\nCASTLEGAR– Effective immediately the Area Re...,https://blog.gov.bc.ca/bcwildfire/area-restric...
3,Area Restriction implemented for Casper Creek ...,1694152800,\nKAMLOOPS – The BC Wildfire Service has imple...,https://blog.gov.bc.ca/bcwildfire/area-restric...
4,Area Restriction expanded for the Downton Lake...,1694152800,\nKAMLOOPS – The BC Wildfire Service has expan...,https://blog.gov.bc.ca/bcwildfire/area-restric...
...,...,...,...,...
185,Open Burning 101 – Category 2 & Category 3,1680242400,\nIn British Columbia it is common for grass f...,https://blog.gov.bc.ca/bcwildfire/open-burning...
186,Prescribed burn for Boitanio Park and Stampede...,1680069600,"\nWILLIAMS LAKE – The BC Wildfire Service, in ...",https://blog.gov.bc.ca/bcwildfire/prescribed-b...
187,Prescribed burn for wildfire reduction along H...,1679896800,\nWILLIAMS LAKE – The BC Wildfire Service in p...,https://blog.gov.bc.ca/bcwildfire/prescribed-b...
188,Pile burns planned for Mill Creek Quarry site,1679637600,\nFORT NELSON—The BC Wildfire Service plans to...,https://blog.gov.bc.ca/bcwildfire/pile-burns-p...
